In [1]:
import json
import pathlib
from tqdm import tqdm

In [28]:
# Construct single json file
data_dir = pathlib.Path('./data/')
json_paths = sorted(data_dir.glob('*.json'))
out_json = pathlib.Path('./merged.json')
data = []
for path in tqdm(json_paths):
    with path.open() as f:
        data.append(json.load(f))
with out_json.open('w') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

100%|██████████| 1266/1266 [00:00<00:00, 8521.40it/s] 


In [33]:
%%bash
# Start solr
../solr-7.3.0/bin/solr start
# Create core
../solr-7.3.0/bin/solr create -c recipe

*** [WARN] *** Your open file limit is currently 1024.  
 It should be set to 65000 to avoid operational disruption. 
 If you no longer wish to see this warning, set SOLR_ULIMIT_CHECKS to false in your profile or solr.in.sh
*** [WARN] ***  Your Max Processes Limit is currently 31295. 
 It should be set to 65000 to avoid operational disruption. 
 If you no longer wish to see this warning, set SOLR_ULIMIT_CHECKS to false in your profile or solr.in.sh
Waiting up to 180 seconds to see Solr running on port 8983 [|]   [/]   [-]   [\]   [|]   [/]   [-]   [\]  
Started Solr server on port 8983 (pid=13340). Happy searching!

    WARNING: Using _default configset with data driven schema functionality. NOT RECOMMENDED for production use.
         To turn off: bin/solr config -c recipe -p 8983 -property update.autoCreateFields -value false

Created new core 'recipe'


In [34]:
%%bash
# Commit
curl 'http://localhost:8983/solr/recipe/update?commit=true' \
    --data-binary @./merged.json \
    -H 'Content-type:application/json'

{
  "responseHeader":{
    "status":0,
    "QTime":2734}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2708k  100    58  100 2708k     21   989k  0:00:02  0:00:02 --:--:--  989k


In [69]:
%%bash

curl http://localhost:8983/solr/recipe/query -d '
{
  "query": "(ingr:onion AND ingr:tomato) (n_ingr:[10 TO 10])",
  "limit": 1,
  "offset": 10,
}'

{
  "responseHeader":{
    "status":0,
    "QTime":1,
    "params":{
      "json":"\n{\n  \"query\": \"(ingr:onion AND ingr:tomato) (n_ingr:[10 TO 10])\",\n  \"limit\": 1,\n  \"offset\": 10,\n}"}},
  "response":{"numFound":300,"start":10,"docs":[
      {
        "slug":["audrey-hepburns-spaghetti-al-pomodoro"],
        "title":["Audrey Hepburn’s Spaghetti Al Pomodoro Recipe by Tasty"],
        "n_step":[7],
        "n_ingr":[10],
        "ingr":["2 tablespoons extra virgin olive oil",
          "1 whole small onion, diced",
          "2 stalks celery, finely diced",
          "2 cloves garlic, minced",
          "1 large bunch fresh basil, washed, half of the leaves minced and half left whole",
          "28 oz (795 g) canned whole peeled tomato, 2 cans",
          "salt, to taste",
          "1 lb (455 g) dried spathetti",
          "freshly grated parmigiano-reggiano cheese",
          "1 cup (125 g) carrots, diced"],
        "step":["dice the carrots, onion, and celery. thinly slice

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3767  100  3670  100    97   398k  10777 --:--:-- --:--:-- --:--:--  408k


In [70]:
%%bash
# Delete core
curl "http://localhost:8983/solr/admin/cores?action=UNLOAD&core=recipe&deleteIndex=true&deleteDataDir=true&deleteInstanceDir=true"
# Shutdown
../solr-7.3.0/bin/solr stop -all

{
  "responseHeader":{
    "status":0,
    "QTime":20}}
Sending stop command to Solr running on port 8983 ... waiting up to 180 seconds to allow Jetty process 13340 to stop gracefully.
    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    56  100    56    0     0   1806      0 --:--:-- --:--:-- --:--:--  1806
